In [117]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from datetime import datetime

In [118]:
# Step 1: Define the risk scores for each protocol

# Load protocol data from CSV
protocols_df = pd.read_csv('protocols.csv')

# Extract Protocol names and Ratings
protocols = protocols_df['Protocol'].tolist()
ratings_values = protocols_df['Rating'].values

np.set_printoptions(precision=10, suppress=True)

ratings_values = np.array(ratings_values)

# Inverted risk scores
inverted_risk_scores = 1 / ratings_values

print(inverted_risk_scores)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [119]:
# Step 2: Create the risk scores matrix (Σ)
Sigma = np.diag(inverted_risk_scores)

In [120]:
# Step 3: Normalize the risk scores matrix (Σ̃)
Sigma_norm = Sigma / np.linalg.norm(Sigma)

In [121]:
print(Sigma_norm)

[[0.0008425705 0.           0.           0.           0.
  0.           0.           0.           0.          ]
 [0.           0.0084684583 0.           0.           0.
  0.           0.           0.           0.          ]
 [0.           0.           0.0027251839 0.           0.
  0.           0.           0.           0.          ]
 [0.           0.           0.           0.013298117  0.
  0.           0.           0.           0.          ]
 [0.           0.           0.           0.           0.1462412554
  0.           0.           0.           0.          ]
 [0.           0.           0.           0.           0.
  0.0425321247 0.           0.           0.          ]
 [0.           0.           0.           0.           0.
  0.           0.0701876959 0.           0.          ]
 [0.           0.           0.           0.           0.
  0.           0.           0.4255690918 0.          ]
 [0.           0.           0.           0.           0.
  0.           0.           0.       

In [122]:
# Step 4: Define the ERC weighting methodology
def risk_contribution(weights, Sigma):
    return np.dot(weights, np.dot(Sigma, weights))

In [123]:
def risk_contributions(weights, Sigma):
    total_risk = risk_contribution(weights, Sigma)
    marginal_contributions = np.dot(Sigma, weights)
    return weights * marginal_contributions / total_risk

In [124]:
def variance_risk_contributions(weights, Sigma):
    contributions = risk_contributions(weights, Sigma)
    return np.var(contributions)

In [125]:
# Step 5: Optimize the Weights

# Number of protocols
n = len(protocols)

# Initial guess: equal weights
initial_weights = np.ones(n) / n  # Initial guess: equal weights

# Constraints: Weights must sum to 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})  # Weights must sum to 1

# Bounds: Weights must be between 0 and 1
bounds = [(0, 1) for _ in range(n)]  # Weights must be between 0 and 1

# Optimize to find the weights that minimize the variance of risk contributions
result = minimize(variance_risk_contributions, initial_weights, args=(Sigma_norm,),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
optimal_weights = result.x

In [126]:
# Step 6: Define the total allocation amount
total_allocation = 10000  # Total amount to be allocated in dollars

In [127]:
# Calculate the dollar allocation for each protocol
dollar_allocation = optimal_weights * total_allocation

In [128]:
# Step 7: Print the summary
summary = []
for protocol, weight, allocation in zip(protocols, optimal_weights, dollar_allocation):
    summary.append({'Protocol': protocol, 'Allocation ($)': allocation, 'Weight (%)': weight * 100})

# Convert the summary to a DataFrame
summary_df = pd.DataFrame(summary)

# Format the DataFrame to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format

summary_df

,Protocol,Allocation ($),Weight (%)
0,aave,3965.48,39.65
1,curve-finance,1249.47,12.49
2,uniswap,2206.21,22.06
3,morpho,991.76,9.92
4,trader-joe,299.09,2.99
5,kamino,557.13,5.57
6,ethena,433.14,4.33
7,stargate,175.74,1.76
8,ramses-exchange,121.99,1.22
